Reference: https://sebastianraschka.com/Articles/2014_python_lda.html

In [1]:
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets

np.random.seed(1)
plt.style.use('classic')

In [2]:
feature_dict = {i:label for i,label in zip(
                range(4),
                  ('sepal length in cm',
                  'sepal width in cm',
                  'petal length in cm',
                  'petal width in cm', ))}

In [3]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [4]:
X.shape, y.shape

((150, 4), (150,))

Promedio de cada caracteristica para cada clase

$$
\begin{align}
\mu_i = \frac{1}{n_i} \sum_{i=1}^{C} x_k
\end{align}
$$

In [ ]:
np.set_printoptions(precision=4)

mean_vectors = []
for category in range(0,3):
    mean_vectors.append(np.mean(X[y==category], axis=0))
    print('Mean Vector class %s: %s\n' %(category, mean_vectors[category]))

Mean Vector class 0: [5.006 3.428 1.462 0.246]

Mean Vector class 1: [5.936 2.77  4.26  1.326]

Mean Vector class 2: [6.588 2.974 5.552 2.026]



In [ ]:
np.array(mean_vectors).shape

(3, 4)

The within class variance (SW)

$$
\begin{align}
S_W = \sum_{i=1}^{C} \sum_{j=0}^{n} x_n \cdot e^{-i2\pi\frac{n}{N}}
\end{align}
$$

In [ ]:
S_W = np.zeros((4,4))
for category, mean_vector in zip(range(1,4), mean_vectors):
    class_scatter_matrix = np.zeros((4,4))
    for row in X[y == category]:
        row, mean_vector = row.reshape(4,1), mean_vector.reshape(4,1)
        class_scatter_matrix += (row-mean_vector).dot((row-mean_vector).T)
    S_W += class_scatter_matrix
print('within-class Scatter Matrix:\n', S_W)

within-class Scatter Matrix:
 [[ 97.3682 -15.1782 196.0494  78.1788]
 [-15.1782  33.6502 -71.3282 -24.0392]
 [196.0494 -71.3282 500.6482 202.2864]
 [ 78.1788 -24.0392 202.2864  88.4324]]


In [30]:
S_W.shape

(4, 4)

The between classes variance (SB)

$$
\begin{align}
S_B = \sum_{i=1}^{C} N_i \left(m_i - m\right) \cdot \left(m_i - m\right)^T
\end{align}
$$

$$
\begin{align}
S_B = \sum_{i=1}^{C} N_i \left(m_i - m\right) \cdot \left(m_i - m\right)^T
\end{align}
$$

In [ ]:
overall_mean = np.mean(X, axis=0)
print(overall_mean)

S_B = np.zeros((4,4))
for i, mean_vector in enumerate(mean_vectors):  
    n = X[y==i+1,:].shape[0]
    mean_vector = mean_vector.reshape(4,1)
    overall_mean = overall_mean.reshape(4,1)
    S_B += n * (mean_vector - overall_mean).dot((mean_vector - overall_mean).T)

print('between-class Scatter Matrix:\n', S_B)

[5.8433 3.0573 3.758  1.1993]
between-class Scatter Matrix:
 [[ 35.4857 -16.8499  98.4518  40.4998]
 [-16.8499  10.9977 -49.7646 -19.4882]
 [ 98.4518 -49.7646 276.181  112.622 ]
 [ 40.4998 -19.4882 112.622   46.2444]]


In [25]:
overall_mean

array([[5.8433],
       [3.0573],
       [3.758 ],
       [1.1993]])

In [28]:
X[y==1+1,:].shape

(50, 4)

Fisher's criterion:

$$
\begin{align}
S_W v = \lambda S_B v
\end{align}
$$

Reformulate as eigen vector with eigen values:
$$
\begin{align}
S_W \cdot S_B^-1 v = \lambda v
\end{align}
$$